# 2b. Combine preprocessed data across runs

Author: Carolyn Parkinson

Date: August 6, 2018

#### Note: Notebooks 2a, 2b and 2c are created just for the purposes of following along and possibly using as a reference in the future, since many of you won't have installed AFNI and since the input files would be very large and would take a long time to download.

After preprocessing, this script concatenates the preprocessed time series data across our runs for each subject, then cleans up their directories a bit.

It takes as input:

- A list of subject IDs (here, *subjects.json*)

- AFNI files for each preprocessed functional run for each subject

And returns:

- *subjectid_epi_all_preprocessed+orig*




In [2]:
import os
import subprocess
import json
import csv
import numpy as np

In [3]:
study_path = "./data"

# load in list of fMRI subjects
with open("{}/fmri/fmri_subjects.json".format(study_path)) as data_file:
    subj_list = json.load(data_file)
    
nruns = 6

In [4]:
def sh(c):
    '''
    run shell commands
    '''
    subprocess.call(c, shell = True)

Concatenate preprocessed data across runs and clean up the subject's data directory a little bit:

In [4]:
for subject in subj_list:

    subj_dir = "{}/fmri/example_subj/{}".format(study_path, subject)
    os.chdir(subj_dir)
    
    
    # copy each run of preprocessed data into the subject's main nifti directory
    epi_fnames = {}
    for epi_num in range(1, nruns+1):
        
        # specify the prefix for the preprocessed file names for this run; this is
        # also the name of the AFNI-generated sub-directory containing these files
        epi_prefix = 'epi{}_preprocessed'.format(epi_num)
        
        # specify file name of preprocessed data within that sub-directory
        epi_fname = '{}.cleanEPI+orig'.format(epi_prefix)
        
        # create a dictionary where  keys are the run names ('epi1', 'epi2', etc.)
        # and values are the corresponding full file names (to iterate through later)
        epi_fnames['epi{}'.format(epi_num)] = epi_fname
        
        # Copy these AFNI files
        cp_cmd = '3dcopy {}/{}/{} {}/{}.cleanEPI'.format(subj_dir, epi_prefix, epi_fname, subj_dir, epi_prefix)
        #sh(cp_cmd)

        
    # Concatenate preprocessed data across runs
    all_epi = "{}_epi_all_preprocessed".format(subject)
    tcat_cmd = ("3dTcat -session {} -prefix {} "
                "{epi1} {epi2} {epi3} {epi4} {epi5} {epi6}").format(subj_dir, all_epi, **epi_fnames)
    sh(tcat_cmd)
 

    # Clean up the subject's directory a bit
    # delete copies of preprocessed data for each run
    for num, fname in epi_fnames.items():
        rm_cmd = 'rm {}/{}*'.format(subj_dir, fname)
        sh(rm_cmd)

    # move raw epi data into its own directory
    raw_epi_dir = '{}/raw_epi_data'.format(subj_dir)
    if not os.path.exists(raw_epi_dir):
        os.makedirs(raw_epi_dir)
    mv_cmd = 'mv {}/{}_fMRI_* {}'.format(subj_dir, subject, raw_epi_dir)
    sh(mv_cmd)